### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6410,2023-11-13,Brasil Nbb,19:30,União Corinthians,Mogi,1.37,3.03,152.5,1.88,1.86,-7.5,2.01,1.72
6411,2023-11-13,Brasil Nbb,20:00,Flamengo,Cerrado,1.01,13.40,155.5,1.86,1.88,-20.5,1.98,1.74
6412,2023-11-13,Brasil Nbb,20:00,Franca,Pinheiros,1.17,4.87,164.5,1.80,1.86,-12.5,2.03,1.70
6413,2023-11-13,Eua Nba,00:00,Los Angeles Lakers,Portland Trail Blazers,1.32,3.93,218.5,1.86,2.07,-8.5,2.06,1.87
6414,2023-11-13,Eua Nba,21:30,Boston Celtics,New York Knicks,1.29,4.13,218.5,1.81,2.06,-9.5,2.04,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6476,2023-11-13,Uruguai Liga Uruguaia,19:15,Goes,Nacional,3.30,1.30,160.5,1.84,1.82,6.5,2.00,1.72
6477,2023-11-13,Uruguai Liga Uruguaia,21:00,Aguada,Larre Borges,1.12,5.52,161.5,1.87,1.79,-13.5,2.06,1.68
6478,2023-11-13,Uruguai Liga Uruguaia,21:00,Bigua,Defensor Sporting,1.79,1.92,158.5,1.86,1.80,-2.5,2.09,1.66
6479,2023-11-13,Uruguai Liga Uruguaia,21:00,Urupan,Trouville,1.59,2.24,161.5,1.84,1.82,-4.5,2.08,1.67


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Los Angeles Lakers,Portland Trail Blazers,218.5,1.86,1.0000,Over
4,21:30,Eua Nba,Boston Celtics,New York Knicks,218.5,1.81,0.9998,Over
5,21:30,Eua Nba,Toronto Raptors,Washington Wizards,225.5,1.80,0.9999,Over
6,22:00,Eua Nba,Milwaukee Bucks,Chicago Bulls,227.5,1.82,1.0000,Over
57,23:30,México Lnbp,Panteras,Libertadores,168.5,1.87,0.7983,Over


### Handicap positivo V4 (LDA)

In [9]:
# Duplicando o dataset
df_hp_v4 = jogos.copy()

df_hp_v4.loc[(df_hp_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v4.loc[(df_hp_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v4['Zebra'] = df_hp_v4['Zebra'].astype('category')

# Odds
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Casa', 'Odds'] = df_hp_v4['HA_Odds_H']
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Fora', 'Odds'] = df_hp_v4['HA_Odds_A']

df_hp_v4.reset_index(drop=True, inplace=True)
df_hp_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v4.empty:
    mdl_hp_v4 = load_model('ML/handicap_positivo/v4/v4_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v4, df_hp_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,20:00,Brasil Nbb,Flamengo,Cerrado,-20.5,1.74,0.9418,HA Fora
10,04:30,Cazaquistão Higher League,Shymkent,Turan,-6.5,1.83,0.5653,HA Fora
14,08:35,China Cba,Ningbo Rockets,Shanghai,2.5,2.08,0.9935,HA Casa
43,14:00,Europa Liga Aba,Krka,Cibona,-5.5,1.71,0.9685,HA Fora
46,17:00,Europa Liga Aba 2,Sencur,Sutjeska,2.5,2.08,0.6206,HA Casa
47,12:15,Grécia Liga De Basquete,AEK Athens,Kolossos Rhodes,-10.5,1.71,0.9743,HA Fora
58,19:30,Paraguai Lnb,Dep. San Jose,Olimpia,-6.5,1.79,0.7177,HA Fora
61,12:00,Rússia Liga Vtb United,Saratov,Zenit Petersburg,6.5,2.02,0.9063,HA Casa
63,14:00,Rússia Liga Vtb United,Runa Basket Moscow,Enisey,-5.5,1.76,0.9691,HA Fora


### Handicap positivo V5 (RF)

In [10]:
# Duplicando o dataset
df_hp_v5 = jogos.copy()

df_hp_v5.loc[(df_hp_v5['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v5.loc[(df_hp_v5['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v5['Zebra'] = df_hp_v5['Zebra'].astype('category')

# Odds
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Casa', 'Odds'] = df_hp_v5['HA_Odds_H']
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Fora', 'Odds'] = df_hp_v5['HA_Odds_A']

df_hp_v5.reset_index(drop=True, inplace=True)
df_hp_v5.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v5.empty:
    mdl_hp_v5 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v5, df_hp_v5)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v5_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,20:00,Brasil Nbb,Flamengo,Cerrado,-20.5,1.74,0.6100,HA Fora
2,20:00,Brasil Nbb,Franca,Pinheiros,-12.5,1.70,0.5600,HA Fora
5,21:30,Eua Nba,Toronto Raptors,Washington Wizards,-10.5,1.83,0.5200,HA Fora
10,04:30,Cazaquistão Higher League,Shymkent,Turan,-6.5,1.83,0.6100,HA Fora
14,08:35,China Cba,Ningbo Rockets,Shanghai,2.5,2.08,0.7000,HA Casa
26,21:00,Eua Ncaa,Miami,Florida Intl,-21.5,1.73,0.5700,HA Fora
43,14:00,Europa Liga Aba,Krka,Cibona,-5.5,1.71,0.6100,HA Fora
44,11:00,Europa Liga Aba 2,Subotica,Borac Banja Luka,-8.5,1.72,0.7700,HA Fora
45,14:00,Europa Liga Aba 2,Podgorica,GKK Sibenka,-10.5,1.72,0.7600,HA Fora
46,17:00,Europa Liga Aba 2,Sencur,Sutjeska,2.5,2.08,0.8200,HA Casa


### Handicap positivo V6 (LDA)

In [11]:
# Duplicando o dataset
df_hp_v6 = jogos.copy()

df_hp_v6.loc[(df_hp_v6['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v6.loc[(df_hp_v6['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v6['Zebra'] = df_hp_v6['Zebra'].astype('category')

# Odds
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Casa', 'Odds'] = df_hp_v6['HA_Odds_H']
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Fora', 'Odds'] = df_hp_v6['HA_Odds_A']

df_hp_v6.reset_index(drop=True, inplace=True)
df_hp_v6.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v6.empty:
    mdl_hp_v6 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v6, df_hp_v6)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v6_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
0,19:30,Brasil Nbb,União Corinthians,Mogi,-7.5,1.72,0.6700,HA Fora
1,20:00,Brasil Nbb,Flamengo,Cerrado,-20.5,1.74,0.6100,HA Fora
2,20:00,Brasil Nbb,Franca,Pinheiros,-12.5,1.70,0.5600,HA Fora
5,21:30,Eua Nba,Toronto Raptors,Washington Wizards,-10.5,1.83,0.5200,HA Fora
9,02:00,Cazaquistão Higher League,ChelSU Kostanay,Atyrau 2,-9.5,1.83,0.7000,HA Fora
10,04:30,Cazaquistão Higher League,Shymkent,Turan,-6.5,1.83,0.6100,HA Fora
11,08:30,Cazaquistão Higher League,Karaganda,Tigers-KazATU,10.5,1.83,0.7000,HA Casa
14,08:35,China Cba,Ningbo Rockets,Shanghai,2.5,2.08,0.7000,HA Casa
15,08:30,China Wcba Feminina,Guangdong F,Sichuan F,9.5,2.01,0.6067,HA Casa
16,08:30,China Wcba Feminina,Shandong F,Shanghai F,1.5,2.08,0.7667,HA Casa
